# Schiefer Wurf mit Luftwiderstand

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

Während in den bisherigen Jupyter-Notebooks eine eindimensionale Bewegung betrachtet wurde, wenden wir uns nun einem zweidimensionalen Problem zu, dem schiefen Wurf mit Luftwiderstand. Da die Newton'sche Reibung quadratisch in der Geschwindigkeit ist, lässt sich das Problem nicht in zwei eindimensionale Probleme separieren wie die dimensionslosen Bewegungsgleichungen

\begin{align}
\frac{\text{d}^2x}{\text{d}t^2} &= -\sqrt{v_x^2+v_z^2} v_x\\
\frac{\text{d}^2z}{\text{d}t^2} &= - 1 - \sqrt{v_x^2+v_z^2} v_z
\end{align}

zeigen. Die im Folgenden angewandten Lösungstechniken entsprechen denen, die in [2-02-Senkrechter-Wurf-mit-Gravitationsgesetz.ipynb](2-02-Senkrechter-Wurf-mit-Gravitationsgesetz.ipynb) verwendet wurden.

### Importanweisungen

Im Vergleich zu den vorhergehenden Jupyter-Notebooks benötigen wir hier vier Funktionen aus dem `math`-Modul der Python-Standardbibliothek. Dabei handelt es sich neben zwei trigonometrischen Funktionen um die Funktion `hypot`, die in zwei Dimensionen die Länge der Hypotenuse eines rechtwinkligen Dreiecks berechnet, wenn die beiden anderen Seitenlängen gegeben sind, sowie die Funktion `radians`, die zur Umwandlung von Grad in Bogenmaß dient. 

In [1]:
from math import cos, hypot, radians, sin
import numpy as np
from scipy import integrate
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

### Implementierung der Bewegungsgleichungen

Nach der Umwandlung der beiden oben angegebenen Differentialgleichungen 2. Ordnung in vier Differentialgleichungen 1. Ordnung ergibt sich die folgende Funktion zur Berechnung der Zeitableitungen, die später von `integrate.solve_ivp` aus der SciPy-Bibliothek benötigt wird.

In [ ]:
def dx_dt(t, x):
    x_x, v_x, x_z, v_z = x
    v = hypot(v_x, v_z)
    return v_x, -v*v_x, v_z, -1-v*v_z

### Abbruchbedingung

Wenn die Punktmasse den Boden in einer Höhe 0 erreicht, soll die Berechnung abgebrochen werden. Die hierfür relevante $z$-Koordinate tritt im vierdimensionalen Vektor gemäß der Funktion `dx_dt` mit dem Index 2 auf. Das Attribut `direction` der Funktion `height_zero` legt fest, dass sich die Punktmasse beim Erreichen des Bodens in der Fallbewegung befindet, so dass die $z$-Koordinate abnimmt. Der Wurf könnte also aus einer Grube mit negativer Anfangshöhe erfolgen, ohne dass die Berechnung in der Steigbewegung abgebrochen würde.

In [ ]:
def height_zero(t, y):
    return y[2]

height_zero.terminal = True
height_zero.direction = -1

### Lösung des Differentialgleichungssystems

Die Bewegungsgleichungen werden mit Hilfe der SciPy-Funktion `integrate.solve_ivp` gelöst, wobei das Vorgehen demjenigen entspricht, das im Jupyter-Notebooks zum senkrechten Wurf im Newton'schen Gravitationspotential zur Anwendung kam.

In [ ]:
def trajectory(t_end, n_out, x0):
    solution = integrate.solve_ivp(
        dx_dt, (0, t_end), x0,
        dense_output=True, events=height_zero)
    if solution.t_events[0].size > 0:
        t_end = solution.t_events[0][0]
    t_values = np.linspace(0, t_end, n_out)
    x_values, _, z_values, _ = solution.sol(t_values)
    return t_values, x_values, z_values

### Implementierung der Bedienelemente und graphische Darstellung der Ergebnisse

Mit den Schiebereglern können die folgenden Variablen eingestellt werden:
- `t_end`: maximaler Zeitraum, über den die Lösung berechnet wird
- `z_0`: Anfangshöhe relativ zum Boden
- `v`: Betrag der Anfangsgeschwindigkeit
- `theta`: Abwurfwinkel bezüglich der Horizontalen in Grad

Innerhalb der Funktion `plot_result` wird festgelegt, dass für die Darstellung der Trajektorie 100 Punkte verwendet werden. Auch dieser Parameter kann bei Bedarf angepasst werden.

In [ ]:
widget_dict = {"t_end":
               widgets.FloatSlider(
                   value=5, min=0.5, max=10, step=0.5,
                   description=r"$\tau_\text{end}$"),
               "z_0":
               widgets.FloatSlider(
                   value=0, min=0, max=2, step=0.1,
                   description="$z_0$"),
               "v":
               widgets.FloatSlider(
                   value=10, min=0, max=30, step=1,
                   description=r"$\vert v\vert$"),
               "theta":
               widgets.FloatSlider(
                   value=45, min=0, max=90, step=5,
                   description=r"$\theta$ (in Grad)")
               }

@interact(**widget_dict)
def plot_result(t_end, z_0, v, theta):
    n_out = 100
    theta_rad = radians(theta)
    x0 = [0, v*cos(theta_rad), z_0, v*sin(theta_rad)]
    t_values, x_values, z_values = trajectory(
        t_end, n_out, x0)

    fig, ax = plt.subplots()
    ax.plot(x_values, z_values)
    ax.set_xlabel("$x$")
    ax.set_ylabel("$z$")